# วันที่ 3: การสร้าง Dashboard และวิเคราะห์กรณีศึกษา
## ช่วงบ่าย (13.00 น.–16.00 น.): กรณีศึกษา – วิเคราะห์ข้อมูลจากสถานการณ์จริง

## 3. การวิเคราะห์ข้อมูลเบื้องต้น

ก่อนที่จะสร้าง Dashboard ใน Power BI เราจะวิเคราะห์ข้อมูลเบื้องต้นเพื่อทำความเข้าใจข้อมูลและหาข้อมูลเชิงลึกที่น่าสนใจ

In [ ]:
# นำเข้าไลบรารีที่จำเป็น
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os

# กำหนดให้แสดงกราฟในโน้ตบุ๊ก
%matplotlib inline
# กำหนดสไตล์ของกราฟ
plt.style.use('seaborn-v0_8')
# กำหนดให้แสดงคอลัมน์ทั้งหมดในดาต้าเฟรม
pd.set_option('display.max_columns', None)

# โหลดข้อมูลที่ได้สร้างไว้
# หากไม่มีไฟล์ CSV ให้รันโค้ดในไฟล์ 'วันที่3-บ่าย-2-การเตรียมข้อมูล.ipynb' ก่อน

try:
    products_df = pd.read_csv('power_bi_data/products.csv')
    customers_df = pd.read_csv('power_bi_data/customers.csv')
    stores_df = pd.read_csv('power_bi_data/stores.csv')
    sales_df = pd.read_csv('power_bi_data/sales.csv')
    
    # แปลงคอลัมน์ที่เกี่ยวกับวันที่ให้เป็น datetime
    sales_df['sale_date'] = pd.to_datetime(sales_df['sale_date'])
    sales_df['month'] = sales_df['sale_date'].dt.strftime('%Y-%m')
    sales_df['quarter'] = sales_df['sale_date'].dt.to_period('Q').astype(str)
    sales_df['year'] = sales_df['sale_date'].dt.year
    sales_df['weekday'] = sales_df['sale_date'].dt.day_name()
    
    # เชื่อมข้อมูลการขายกับข้อมูลอื่นๆ
    sales_with_info = sales_df.merge(products_df, on='product_id', how='left')
    sales_with_info = sales_with_info.merge(customers_df, on='customer_id', how='left')
    sales_with_info = sales_with_info.merge(stores_df, on='store_id', how='left')
    
    print("โหลดข้อมูลเรียบร้อยแล้ว!")
except FileNotFoundError:
    print("ไม่พบไฟล์ข้อมูล โปรดรันโค้ดในไฟล์ 'วันที่3-บ่าย-2-การเตรียมข้อมูล.ipynb' ก่อน")

### 3.1 การวิเคราะห์ยอดขายและกำไร

In [ ]:
# สรุปยอดขายและกำไรโดยรวม
total_sales = sales_df['sales_amount'].sum()
total_profit = sales_df['profit'].sum()
profit_margin = (total_profit / total_sales) * 100
total_transactions = sales_df['transaction_id'].nunique()
avg_transaction_value = total_sales / total_transactions

print(f"ยอดขายรวม: {total_sales:,.2f} บาท")
print(f"กำไรรวม: {total_profit:,.2f} บาท")
print(f"อัตรากำไร: {profit_margin:.2f}%")
print(f"จำนวนธุรกรรมทั้งหมด: {total_transactions:,}")
print(f"มูลค่าเฉลี่ยต่อธุรกรรม: {avg_transaction_value:,.2f} บาท")

In [ ]:
# วิเคราะห์ยอดขายตามหมวดหมู่สินค้า
sales_by_category = sales_with_info.groupby('category').agg({
    'sales_amount': 'sum',
    'profit': 'sum',
    'transaction_id': pd.Series.nunique
}).reset_index()
sales_by_category['profit_margin'] = (sales_by_category['profit'] / sales_by_category['sales_amount']) * 100
sales_by_category = sales_by_category.sort_values('sales_amount', ascending=False)

print("ยอดขายและกำไรตามหมวดหมู่สินค้า:")
display(sales_by_category)

# แสดงกราฟแท่งเปรียบเทียบยอดขายตามหมวดหมู่สินค้า
plt.figure(figsize=(10, 6))
sns.barplot(x='category', y='sales_amount', data=sales_by_category)
plt.title('ยอดขายตามหมวดหมู่สินค้า')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# วิเคราะห์อัตรากำไรตามหมวดหมู่สินค้า
plt.figure(figsize=(10, 6))
sns.barplot(x='category', y='profit_margin', data=sales_by_category)
plt.title('อัตรากำไรตามหมวดหมู่สินค้า')
plt.xticks(rotation=45)
plt.ylabel('อัตรากำไร (%)')
plt.tight_layout()
plt.show()

In [ ]:
# วิเคราะห์แนวโน้มยอดขายรายเดือน
monthly_sales = sales_df.groupby('month').agg({
    'sales_amount': 'sum',
    'profit': 'sum',
    'transaction_id': pd.Series.nunique
}).reset_index()
monthly_sales = monthly_sales.sort_values('month')

print("แนวโน้มยอดขายและกำไรรายเดือน:")
display(monthly_sales.head())

# แสดงกราฟเส้นแนวโน้มยอดขายรายเดือน
plt.figure(figsize=(12, 6))
plt.plot(monthly_sales['month'], monthly_sales['sales_amount'], marker='o', linewidth=2, label='ยอดขาย')
plt.plot(monthly_sales['month'], monthly_sales['profit'], marker='o', linewidth=2, label='กำไร')
plt.title('แนวโน้มยอดขายและกำไรรายเดือน')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# วิเคราะห์ยอดขายและกำไรตามไตรมาส
quarterly_sales = sales_df.groupby('quarter').agg({
    'sales_amount': 'sum',
    'profit': 'sum',
    'transaction_id': pd.Series.nunique
}).reset_index()
quarterly_sales['profit_margin'] = (quarterly_sales['profit'] / quarterly_sales['sales_amount']) * 100
quarterly_sales = quarterly_sales.sort_values('quarter')

print("ยอดขายและกำไรตามไตรมาส:")
display(quarterly_sales)

# แสดงกราฟแท่งเปรียบเทียบยอดขายตามไตรมาส
plt.figure(figsize=(10, 6))
x = range(len(quarterly_sales))
width = 0.35

plt.bar(x, quarterly_sales['sales_amount'], width, label='ยอดขาย')
plt.bar([i + width for i in x], quarterly_sales['profit'], width, label='กำไร')

plt.xlabel('ไตรมาส')
plt.ylabel('จำนวนเงิน (บาท)')
plt.title('ยอดขายและกำไรตามไตรมาส')
plt.xticks([i + width/2 for i in x], quarterly_sales['quarter'])
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# วิเคราะห์อัตรากำไรและจำนวนธุรกรรมตามไตรมาส
fig, ax1 = plt.subplots(figsize=(10, 6))

color = 'tab:blue'
ax1.set_xlabel('ไตรมาส')
ax1.set_ylabel('อัตรากำไร (%)', color=color)
ax1.plot(quarterly_sales['quarter'], quarterly_sales['profit_margin'], color=color, marker='o', linewidth=2)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # สร้างแกน y ที่สอง
color = 'tab:red'
ax2.set_ylabel('จำนวนธุรกรรม', color=color)
ax2.plot(quarterly_sales['quarter'], quarterly_sales['transaction_id'], color=color, marker='s', linewidth=2)
ax2.tick_params(axis='y', labelcolor=color)

plt.title('อัตรากำไรและจำนวนธุรกรรมตามไตรมาส')
fig.tight_layout()
plt.show()